# 05-04: Small-world networks

*November 24 2021*

In the final unit of this week we study properties of synthetic and empirical small-world networks and show how we can visualize the property of *shortest path funnelling* using `pathpy`.

In [1]:
import pathpy as pp
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('default')
sns.set_style("whitegrid")

# Shortest Paths in random vs. small-world networks

In the previous practice session, we have seen that many empirical social networks combine a small diameter (or even a small average shortest path length) with a large clustering coefficient, the latter of which we can cannot explain with a simple random graph model. In the lecture, we have further discussed Milgram's experiment, which shows that humans are able to efficiently "navigate" social networks without having global knowledge. To better understand how this works, we have introduced the concept of path funnelling. Let us implement a function that can visualize this in a given network.

In [3]:
def funelling_plot(n, style={}, layout=None, source=None):
    # visualise number of shortest paths from given node x to all other nodes, passing through neighbors of x
    if source is None:
        source = np.random.choice(list(n.nodes.uids))
        print(source)
    labels = { v: v for v in n.nodes.uids }
    style['node_label_size'] = { v: 2 for v in n.nodes.uids}
    style['node_size'] = { v: 20 for v in n.nodes.uids}
    style['node_color'] = { v: 'CornflowerBlue' for v in n.nodes.uids }
    style['node_color'][source] = 'Red'

    tree = pp.algorithms.shortest_path_tree(n, source=source)
    assert pp.algorithms.check_tree(tree)

    # compute tree size for all neighbors of source
    for v in n.successors[source]:    
        s = pp.algorithms.tree_size(tree, v.uid)
        labels[v.uid] = v.uid + ': ' + str(s)
        style['node_label_size'][v.uid] = 2 + 0.1*s
        style['node_size'][v.uid] = 5 + 2*s
        style['node_color'][v.uid] = 'Green'
    if layout is None:
        layout = pp.layout(n, layout='fr', force=0.5)
    
    n.plot(node_label=labels, node_id_as_label=False, **style, layout=layout)

For a given source node (shown in red), the function above shows how many shortest paths from this source node to any other node pass through the neighbors of the source node. Let us try this in a Watts-Strogatz network where we rewire all edges with probably 1 (which yields a random graph that is - although not identical to - virtually indistinguishable from a random graph).

In [10]:
style = {}
style['edge_directed'] = False
style['edge_color'] = 'gray'
style['node_color'] = 'CornflowerBlue'
style['node_label_size'] = 2
style['node_size'] = 20
style['edge_size'] = 1.0

n_random = pp.generators.Watts_Strogatz(n=50, s=2, p=1, loops=False)
funelling_plot(n_random, style)

18


In most realizations, the number of shortest paths passing through each neighbour will be rather similar. This is different for a two-dimensional lattice network with lattice parameter $s>1$. Here two neighbors stand out, as they lie on almost all shortest paths to other nodes. Note that the symmetry is broken due to the odd number of *other* nodes in the network, so there are more nodes on one side of the coordinate space than on the other side.

In [13]:
n_ring = pp.generators.Watts_Strogatz(n=50, s=2, p=0)
layout = pp.layout(n_ring, layout='lattice', dimension=1)
funelling_plot(n_ring, style, layout)

3


If we increase the rewiring probability of the Watts-Strogatz model, this heterogeneity of neighbors is preserved, while -- due to the potential shortcuts introduced with each rewired edge -- the diameter and average shortest path lengths quickly decrease. The resulting funnelling plot now reveals that, in most realizations, there is a clear choice which neighbor we should prefer to contact if we are interested to find a shortest path to any other node in the network.

In [16]:
n_ws = pp.generators.Watts_Strogatz(n=50, s=4, p=0.075)
funelling_plot(n_ws, style)

32


# Empirical small-world networks

We finally test for which of the three empirical networks we find a small-world property. For this, we compare their average shortest path lengths and clustering coefficients to that of random networks with the same macrostate. If the average shortest path lengths are similar to that of a random network (or even smaller) and the clustering coefficient is much larger than expected at random, we call the network a small-world network. 

In [17]:
n_gentoo = pp.io.sql.read_network('../data/networks.db', sql='SELECT DISTINCT source, target FROM gentoo', directed=False, loops=False).largest_connected_component()
n_highschool = pp.io.sql.read_network('../data/networks.db', sql='SELECT DISTINCT source, target FROM highschool', directed=False, loops=False).largest_connected_component()
n_lotr = pp.io.sql.read_network('../data/networks.db', sql='SELECT DISTINCT source, target FROM lotr', directed=False, loops=False).largest_connected_component()

[11-23 17:04:33: WARNING] 154 edges existed already and were not be considered. To capture those edges, consider creating a multiedge and/or directed network.
[11-23 17:04:33: WARNING] 67 edges existed already and were not be considered. To capture those edges, consider creating a multiedge and/or directed network.


In [18]:
r_gentoo = pp.generators.ER_np_randomize(n_gentoo).largest_connected_component()

print('cc_e = ', pp.statistics.avg_clustering_coefficient(n_gentoo))
print('cc_r = ', pp.statistics.avg_clustering_coefficient(r_gentoo))
print('<l_e> = ', pp.algorithms.avg_path_length(n_gentoo))
print('<l_r> = ', pp.algorithms.avg_path_length(r_gentoo))

cc_e =  0.017357355654577637
cc_r =  0.0050450450450450456
<l_e> =  3.1096379146451367
<l_r> =  6.079381820845235


In [19]:
r_highschool = pp.generators.ER_np_randomize(n_highschool).largest_connected_component()

print('cc_e = ', pp.statistics.avg_clustering_coefficient(n_highschool))
print('cc_r = ', pp.statistics.avg_clustering_coefficient(r_highschool))
print('<l_e> = ', pp.algorithms.avg_path_length(n_highschool))
print('<l_r> = ', pp.algorithms.avg_path_length(r_highschool))

cc_e =  0.446799543049543
cc_r =  0.034662698412698416
<l_e> =  5.362745098039215
<l_r> =  3.026050420168067


In [20]:
r_lotr = pp.generators.ER_np_randomize(n_lotr).largest_connected_component()

print('cc_e = ', pp.statistics.avg_clustering_coefficient(n_lotr))
print('cc_r = ', pp.statistics.avg_clustering_coefficient(r_lotr))
print('<l_e> = ', pp.algorithms.avg_path_length(n_lotr))
print('<l_r> = ', pp.algorithms.avg_path_length(r_lotr))

cc_e =  0.5848854454659764
cc_r =  0.07282241707175861
<l_e> =  2.6724273369992146
<l_r> =  2.399169565705308


How does the small-world property of the Lord of the Rings character network translate to funnelling? To check this, we can create the shortest path funnelling plot for Samwise Gamgee, which reveals that Gandalfs and Aragon are the two neighbors through which on a majority of the shortest paths from Sam to any other node. 

In [21]:
funelling_plot(n_lotr, source='Sam')